In [1]:
"""
combine_stabl_features.py
-------------------------
Outils pour combiner les listes de variables sélectionnées par plusieurs
méthodes STABL (Lasso, XGBoost, RandomForest, …).

Usage
-----
# 1) depuis la RAM (selected_features_dict déjà construit) :
from combine_stabl_features import combine_features

combo = combine_features(
    selected_features_dict,
    base_models=["STABL Lasso", "STABL XGBoost"],
    strategy="union",          # "union", "intersection" ou "vote"
    vote_k=2,                  # utilisé si strategy="vote"
    save_path="./combo_features"
)

# combo["per_fold"]  -> dict {fold_i: [var1, var2, ...]}
# combo["global"]    -> set(...) union / intersection / vote
# combo["summary"]   -> DataFrame (taille par fold)
--------------------------------------------------------------------
# 2) pour charger automatiquement les CSV générés par votre pipeline :
from combine_stabl_features import load_selected_from_csv, combine_features

sel_feats = load_selected_from_csv(
    csv_dir="./Benchmarks results/Binary COVID-19/Training CV",
    models=["STABL Lasso", "STABL XGBoost"]
)

combo = combine_features(sel_feats, ["STABL Lasso", "STABL XGBoost"], "union")
"""

import os
import re
import pandas as pd
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Literal, Set

# ------------------------------------------------------------------
# 1) Charger les CSV “Selected Features xxx.csv” produits par la CV
# ------------------------------------------------------------------
def load_selected_from_csv(
    csv_dir: str | Path,
    models: List[str]
) -> Dict[str, List[List[str]]]:
    """
    Lit les CSV `Selected Features {model}.csv` exportés par le pipeline.

    Parameters
    ----------
    csv_dir  : dossier qui contient les CSV (ex. "./Benchmarks results/.../Training CV")
    models   : liste des modèles STABL à charger (les clés du dict retourné)

    Returns
    -------
    dict : {model_name: [list_fold1, list_fold2, ...]}
    """
    csv_dir = Path(csv_dir)
    sel_dict = {}
    for m in models:
        csv_file = csv_dir / f"Selected Features {m}.csv"
        if not csv_file.exists():
            raise FileNotFoundError(csv_file)
        df = pd.read_csv(csv_file, index_col=0)
        # la colonne "Fold selected features" contient des chaînes style "['F1', 'F2']"
        lists = df["Fold selected features"].apply(eval).tolist()
        sel_dict[m] = lists
    return sel_dict


# ------------------------------------------------------------------
# 2) Combiner plusieurs listes selon union / intersection / vote
# ------------------------------------------------------------------
def combine_features(
    selected_features_dict: Dict[str, List[List[str]]],
    base_models: List[str],
    strategy: Literal["union", "intersection", "vote"] = "union",
    vote_k: int = 2,
    save_path: str | Path | None = None,
) -> Dict[str, object]:
    """
    Combine les listes sélectionnées par plusieurs modèles STABL.

    Parameters
    ----------
    selected_features_dict : dict {model -> [list_fold1, list_fold2, ...]}
    base_models            : sous-ensemble de clés du dict à combiner
    strategy               : "union", "intersection", "vote"
    vote_k                 : seuil pour la stratégie "vote" (>= k sélections)
    save_path              : dossier CSV de sortie (optionnel)

    Returns
    -------
    dict avec trois clés :
        - "per_fold" : {fold_i: [var1, var2, ...]}
        - "global"   : set(...)  (union / intersection / vote global)
        - "summary"  : DataFrame  (#variables par fold)
    """

    # Vérification
    for m in base_models:
        if m not in selected_features_dict:
            raise KeyError(f"{m} absent de selected_features_dict")

    n_folds = len(next(iter(selected_features_dict.values())))

    # 2.1) Préparer un compteur par fold pour la stratégie "vote"
    vote_counter = [defaultdict(int) for _ in range(n_folds)]

    # 2.2) Résultat par fold
    per_fold: Dict[str, List[str]] = {}

    for fold_idx in range(n_folds):
        lists_fold = [set(selected_features_dict[m][fold_idx]) for m in base_models]

        if strategy == "union":
            comb = set().union(*lists_fold)

        elif strategy == "intersection":
            comb = set.intersection(*lists_fold) if lists_fold else set()

        elif strategy == "vote":
            # comptage
            for s in lists_fold:
                for feat in s:
                    vote_counter[fold_idx][feat] += 1
            comb = {feat for feat, cnt in vote_counter[fold_idx].items() if cnt >= vote_k}

        else:
            raise ValueError("strategy must be 'union', 'intersection' or 'vote'")

        per_fold[f"Fold {fold_idx+1}"] = sorted(comb)

    # 2.3) Combinaison globale (across folds)
    if strategy == "union":
        global_set: Set[str] = set().union(*per_fold.values())

    elif strategy == "intersection":
        global_set = set.intersection(*map(set, per_fold.values()))

    elif strategy == "vote":
        global_counter = defaultdict(int)
        for fold_list in per_fold.values():
            for feat in fold_list:
                global_counter[feat] += 1
        global_set = {feat for feat, cnt in global_counter.items() if cnt >= vote_k}

    # 2.4) Tableau récap
    summary = pd.DataFrame({
        "n_features": [len(v) for v in per_fold.values()]
    }, index=per_fold.keys())

    # 2.5) Sauvegarde CSV
    if save_path is not None:
        save_path = Path(save_path)
        save_path.mkdir(parents=True, exist_ok=True)

        # par fold
        for fold, lst in per_fold.items():
            pd.Series(lst, name="features").to_csv(save_path / f"combined_{strategy}_{fold}.csv", index=False)

        # global
        pd.Series(sorted(global_set), name="features").to_csv(save_path / f"combined_{strategy}_GLOBAL.csv", index=False)

        # summary
        summary.to_csv(save_path / f"combined_{strategy}_SUMMARY.csv")

    return {
        "per_fold": per_fold,
        "global": global_set,
        "summary": summary
    }


In [7]:

# --- 1) charger les CSV STABL (adapter le chemin + modèles) ----
stabl_models = ["STABL RandomForest", "STABL XGBoost"]
sel_dict = load_selected_from_csv(
    csv_dir="/Users/noeamar/Documents/Stanford/Benchmarks results/Binary COVID-19 + RF/KO/Training CV",
    models=stabl_models
)

# --- 2) faire l’union des listes -------------------------------
combo = combine_features(
    selected_features_dict=sel_dict,
    base_models=stabl_models,
    strategy="union",      # ou "intersection" / "vote"
    vote_k=2,
    save_path="./Benchmarks results/Binary COVID-19 + RF/KO/Training CV/Combined Features"
)

print("➡  Global combined set :", len(combo["global"]), "variables")
print("➡  Summary\n", combo["summary"])

➡  Global combined set : 25 variables
➡  Summary
          n_features
Fold 1            8
Fold 2            6
Fold 3            8
Fold 4            2
Fold 5            7
Fold 6            3
Fold 7            9
Fold 8            7
Fold 9            2
Fold 10           4
Fold 11           3
Fold 12           6
Fold 13           5
Fold 14           3
Fold 15           4


In [8]:
from pathlib import Path
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

def rf_on_combined_features(
    X_tot: pd.DataFrame,
    y: pd.Series,
    outer_splitter,
    combined_dir: str | Path,
    model_name: str = "RF on Combined",
    rf_params: dict | None = None,
    save_dir: str | Path | None = None,
):
    """
    Ré-évalue un RandomForest sur les variables combinées par fold.

    Parameters
    ----------
    X_tot, y        : jeu complet (colonnes concaténées)
    outer_splitter  : le même splitteur CV que pour STABL
    combined_dir    : dossier où se trouvent les CSV
                      "combined_*_Fold 1.csv", "combined_*_Fold 2.csv", ...
    model_name      : étiquette pour les fichiers de sortie
    rf_params       : dict d'hyper-paramètres RF (None ➜ valeurs par défaut)
    save_dir        : si non-None, dossier pour sauvegarder prédictions & résumé

    Returns
    -------
    preds_median : pd.Series (médiane des folds par échantillon)
    preds_folds  : pd.DataFrame (probas par fold, index = y.index)
    summary_df   : pd.DataFrame (nb de variables par fold)
    """
    combined_dir = Path(combined_dir)
    fold_files = sorted(combined_dir.glob("combined_*_Fold *.csv"),
                        key=lambda p: int(p.stem.split()[-1]))  # assure l'ordre 1,2,3...

    if not fold_files:
        raise FileNotFoundError(f"Aucun CSV 'combined_*_Fold X.csv' dans {combined_dir}")

    # Prépare les listes par fold
    combined_lists = {}
    for f in fold_files:
        fold_num = int(f.stem.split()[-1])      # dernier token = numéro de fold
        combined_lists[f"Fold {fold_num}"] = pd.read_csv(f)["features"].tolist()

    # Config RF par défaut
    if rf_params is None:
        rf_params = dict(n_estimators=500, max_features=0.2, random_state=42)

    rf_clf = RandomForestClassifier(**rf_params)

    prepro = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("std",     StandardScaler())
    ])

    # DataFrame pour stocker proba classe 1 par fold
    preds_folds = pd.DataFrame(index=y.index)
    n_features_fold = {}

    k = 1
    for train_idx, test_idx in outer_splitter.split(X_tot, y):
        fold_key = f"Fold {k}"
        feats = combined_lists.get(fold_key, [])

        if len(feats) == 0:
            # aucune variable ➜ prédiction neutre
            preds_folds.loc[y.iloc[test_idx].index, fold_key] = 0.5
            n_features_fold[fold_key] = 0
            k += 1
            continue

        X_train = X_tot.loc[y.iloc[train_idx].index, feats]
        X_test  = X_tot.loc[y.iloc[test_idx].index,  feats]
        y_train = y.iloc[train_idx]

        X_train_std = prepro.fit_transform(X_train)
        X_test_std  = prepro.transform(X_test)

        preds = (clone(rf_clf)
                 .fit(X_train_std, y_train)
                 .predict_proba(X_test_std)[:, 1])

        preds_folds.loc[y.iloc[test_idx].index, fold_key] = preds
        n_features_fold[fold_key] = len(feats)
        k += 1

    # Médiane des folds
    preds_median = preds_folds.median(axis=1)

    # Résumé par fold
    summary_df = pd.DataFrame({
        "n_features": pd.Series(n_features_fold)
    })

    # Sauvegardes optionnelles
    if save_dir is not None:
        save_dir = Path(save_dir)
        save_dir.mkdir(parents=True, exist_ok=True)
        preds_folds.to_csv(save_dir / f"{model_name}_preds_per_fold.csv")
        preds_median.to_csv(save_dir / f"{model_name}_median.csv", header=[model_name])
        summary_df.to_csv(save_dir / f"{model_name}_summary.csv")

    return preds_median, preds_folds, summary_df


In [12]:
"""
benchmark_union_intersection.py
--------------------------------
Évalue RF, XGB et Logit sur des *combinaisons de features* (union / intersection)
issues de plusieurs sélecteurs STABL, sans toucher à la pipeline originale.

Dossier attendu :
└── Training CV/
    ├── Selected Features STABL Lasso.csv
    ├── Selected Features STABL RandomForest.csv
    └── Selected Features STABL XGBoost.csv
"""
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from stabl import data
from xgboost import XGBClassifier

import numpy as np

def median_ci(values, confidence_level: float = 0.95) -> str:
    """
    Renvoie une chaîne "médiane [borne-basse, borne-haute]" pour
    n’importe quel tableau de scores (ROC, AP, #features, etc.).

    Parameters
    ----------
    values            : liste ou array de nombres (peut contenir des NaN)
    confidence_level  : niveau de confiance pour l’IC (par défaut 95 %)

    Example
    -------
    >>> median_ci([0.81, 0.79, 0.85, 0.84])
    '0.825 [0.792, 0.847]'
    """
    vals = np.asarray(values, dtype=float)
    med  = np.nanmedian(vals)
    lo, hi = np.nanpercentile(
        vals,
        [(1 - confidence_level) / 2 * 100,
         (1 + confidence_level) / 2 * 100]
    )
    return f"{med:.3f} [{lo:.3f}, {hi:.3f}]"


# ------------------------------------------------------------------
# 0) ▸ PARAMÈTRES  (à ajuster UNE fois)
# ------------------------------------------------------------------
DATA_DIR = Path("/Users/noeamar/Documents/Stanford/Benchmarks results/"
                "Binary COVID-19 + RF/KO/Training CV")
SELECTED_DIR = DATA_DIR         # là où sont les “Selected Features …”
SAVE_DIR     = DATA_DIR / "Union_Inter_section_Benchmark"

COMBOS = [                       # toutes les combinaisons à tester
    ("STABL Lasso", "STABL XGBoost"),
    ("STABL RandomForest", "STABL Lasso"),
    ("STABL RandomForest", "STABL XGBoost"),
]

STRATEGIES = ["union", "intersection"]  # ou ["union"] si tu ne veux qu’une seule

CLF_DICT = {                     # modèles finaux
    "RF":  RandomForestClassifier(n_estimators=500, max_features=0.2, random_state=42),
    "XGB": XGBClassifier(n_estimators=200, use_label_encoder=False,
                         eval_metric="logloss", random_state=42),
    "Logit": LogisticRegression(penalty="l1", solver="liblinear",
                                class_weight="balanced", max_iter=int(1e6), random_state=42)
}

# ------------------------------------------------------------------
# 1) ▸ CHARGER les Selected-Features par modèle / par fold
# ------------------------------------------------------------------
def load_selected(model):
    f = SELECTED_DIR / f"Selected Features {model}.csv"
    df = pd.read_csv(f, index_col=0)
    return df["Fold selected features"].apply(eval).tolist()   # -> list of lists

selected_by_model = {m: load_selected(m) for m in {m for c in COMBOS for m in c}}
N_FOLDS = len(next(iter(selected_by_model.values())))          # 15 si 5×3

# ------------------------------------------------------------------
# 2) ▸ FONCTION de combinaison par fold
# ------------------------------------------------------------------
def combine_lists(l1, l2, strategy):
    if strategy == "union":
        return list(set(l1) | set(l2))
    elif strategy == "intersection":
        return list(set(l1) & set(l2))
    else:
        raise ValueError

# ------------------------------------------------------------------
# 3) ▸ PRÉPARER X_tot, y, outer_cv  (identiques à la pipeline)
# ------------------------------------------------------------------
X_dict, _, y, _, _, _ = data.load_covid_19("data/COVID-19")
X_tot = pd.concat(X_dict.values(), axis=1)
outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

# ------------------------------------------------------------------
# 4) ▸ BOUCLE : pour chaque combo, strategy, classifieur
# ------------------------------------------------------------------
prepro = Pipeline([("imputer", SimpleImputer(strategy="median")),
                   ("std",     StandardScaler())])

big_scores = []

for mdl1, mdl2 in COMBOS:
    lists1, lists2 = selected_by_model[mdl1], selected_by_model[mdl2]

    for strat in STRATEGIES:
        # --- combine per fold ---
        comb_per_fold = {
            f"Fold {i+1}": combine_lists(l1, l2, strat)
            for i, (l1, l2) in enumerate(zip(lists1, lists2))
        }

        for clf_name, base_est in CLF_DICT.items():
            tag = f"{strat.capitalize()}({mdl1}+{mdl2}) + {clf_name}"
            print(f"\n### {tag}")

            preds_folds = pd.DataFrame(index=y.index)
            metrics = []

            k = 1
            for tr, te in outer_cv.split(X_tot, y):
                feats = comb_per_fold[f"Fold {k}"]
                idx_tr, idx_te = y.iloc[tr].index, y.iloc[te].index

                if len(feats) == 0:
                    preds_folds.loc[idx_te, f"Fold {k}"] = 0.5
                    metrics.append((np.nan, np.nan, 0))
                    k += 1; continue

                X_tr = prepro.fit_transform(X_tot.loc[idx_tr, feats])
                X_te = prepro.transform( X_tot.loc[idx_te, feats])
                y_tr, y_te = y.loc[idx_tr], y.loc[idx_te]

                est = clone(base_est).fit(X_tr, y_tr)
                proba = (est.predict_proba(X_te)[:, 1]
                         if hasattr(est, "predict_proba")
                         else est.decision_function(X_te))

                preds_folds.loc[idx_te, f"Fold {k}"] = proba
                roc = roc_auc_score(y_te, proba)
                ap  = average_precision_score(y_te, proba)
                metrics.append((roc, ap, len(feats)))
                k += 1

            # ---- résumé ----
            roc_ci   = median_ci([m[0] for m in metrics])
            ap_ci    = median_ci([m[1] for m in metrics])
            nfeat_ci = median_ci([m[2] for m in metrics])
            cvs_ci   = median_ci(preds_folds.std(axis=1) /
                                 preds_folds.mean(axis=1).replace(0, np.nan))

            big_scores.append([tag, roc_ci, ap_ci, nfeat_ci, cvs_ci])

            # ---- sauvegarde fichiers ----
            out = SAVE_DIR / tag.replace(" ", "_")
            out.mkdir(parents=True, exist_ok=True)
            preds_folds.to_csv(out / "preds_per_fold.csv")
            preds_folds.median(axis=1).to_csv(out / "median.csv", header=[tag])
            pd.DataFrame({"n_features": [len(f) for f in comb_per_fold.values()]},
                         index=comb_per_fold.keys()).to_csv(out / "summary.csv")

# ------------------------------------------------------------------
# 5) ▸ Écrire / compléter Scores training CV.csv
# ------------------------------------------------------------------
score_df = pd.DataFrame(big_scores,
                        columns=["Model", "ROC AUC", "Average Precision",
                                 "N features", "CVS"]).set_index("Model")
score_csv = DATA_DIR / "Scores training CV.csv"
score_df = (pd.read_csv(score_csv, index_col=0)
            .combine_first(score_df) if score_csv.exists() else score_df)
score_df.to_csv(score_csv)

print("\n✅  Résultats ajoutés à :", score_csv)



### Union(STABL Lasso+STABL XGBoost) + RF

### Union(STABL Lasso+STABL XGBoost) + XGB


/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:17:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:17:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:17:57] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:17:57] WARNING: /Users/runner/work/xgboost/xgboost/src/lear


### Union(STABL Lasso+STABL XGBoost) + Logit

### Intersection(STABL Lasso+STABL XGBoost) + RF

### Intersection(STABL Lasso+STABL XGBoost) + XGB


/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:17:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:17:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:17:59] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:18:00] WARNING: /Users/runner/work/xgboost/xgboost/src/lear


### Intersection(STABL Lasso+STABL XGBoost) + Logit

### Union(STABL RandomForest+STABL Lasso) + RF

### Union(STABL RandomForest+STABL Lasso) + XGB


...


### Intersection(STABL RandomForest+STABL Lasso) + Logit

### Union(STABL RandomForest+STABL XGBoost) + RF

### Union(STABL RandomForest+STABL XGBoost) + XGB


/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:18:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:18:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:18:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:18:08] WARNING: /Users/runner/work/xgboost/xgboost/src/lear


### Union(STABL RandomForest+STABL XGBoost) + Logit

### Intersection(STABL RandomForest+STABL XGBoost) + RF

### Intersection(STABL RandomForest+STABL XGBoost) + XGB


/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:18:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:18:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:18:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/noeamar/Documents/Stanford/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [10:18:12] WARNING: /Users/runner/work/xgboost/xgboost/src/lear


### Intersection(STABL RandomForest+STABL XGBoost) + Logit

✅  Résultats ajoutés à : /Users/noeamar/Documents/Stanford/Benchmarks results/Binary COVID-19 + RF/KO/Training CV/Scores training CV.csv
